<a href="https://colab.research.google.com/github/jmestanza/time_stretching/blob/master/code/vad_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
%cd /content
from IPython.display import clear_output
#!git clone https://github.com/jmestanza/time_stretching.git

#imports
!pip install webrtcvad
import webrtcvad
#!pip install librosa
!pip install --upgrade librosa
import librosa 
import scipy
from IPython.display import Audio

/content
Requirement already up-to-date: librosa in /usr/local/lib/python3.6/dist-packages (0.7.2)


In [0]:
#local modules
!wget https://raw.githubusercontent.com/jmestanza/time_stretching/master/LPC%2C%20PSOLA%20and%20Pitch%20detection%20speed%20up/detect_pitch.py
!wget https://raw.githubusercontent.com/jmestanza/time_stretching/master/LPC%2C%20PSOLA%20and%20Pitch%20detection%20speed%20up/modified_psola.py

from modified_psola import modified_psola
from detect_pitch import get_fundamental_frequency

#audios
!wget "https://raw.githubusercontent.com/jmestanza/time_stretching/master/LPC%2C%20PSOLA%20and%20Pitch%20detection%20speed%20up/input/ahh_without_silences_8k.wav"
!wget "https://raw.githubusercontent.com/jmestanza/time_stretching/master/LPC%2C%20PSOLA%20and%20Pitch%20detection%20speed%20up/input/ahh_without_silences_8k_norm.wav"

clear_output()

In [0]:
import numpy as np

In [0]:
import sys
import array
import struct
import wave

def convert(fin, fout, chunk_size = 1024 * 1024):
    chunk_size *= 4    # convert from samples to bytes

    waveout = wave.open(fout, "wb")
    waveout.setparams((1, 2, 44100, 0, "NONE", ""))

    while True:
        raw_floats = fin.read(chunk_size)
        if raw_floats == "":
            return
        floats = array.array('f', raw_floats)
        samples = [sample * 32767
                   for sample in floats]
        raw_ints = struct.pack("<%dh" % len(samples), *samples)
        waveout.writeframes(raw_ints)

#convert(open(sys.argv[1], "rb"), open(sys.argv[2], "wb"))

In [0]:
#https://github.com/mgeier/python-audio/blob/master/audio-files/utility.py
def float2pcm(sig, dtype='int16'):
    """Convert floating point signal with a range from -1 to 1 to PCM.
    Any signal values outside the interval [-1.0, 1.0) are clipped.
    No dithering is used.
    Note that there are different possibilities for scaling floating
    point numbers to PCM numbers, this function implements just one of
    them.  For an overview of alternatives see
    http://blog.bjornroche.com/2009/12/int-float-int-its-jungle-out-there.html
    Parameters
    ----------
    sig : array_like
        Input array, must have floating point type.
    dtype : data type, optional
        Desired (integer) data type.
    Returns
    -------
    numpy.ndarray
        Integer data, scaled and clipped to the range of the given
        *dtype*.
    See Also
    --------
    pcm2float, dtype
    """
    sig = np.asarray(sig)
    if sig.dtype.kind != 'f':
        raise TypeError("'sig' must be a float array")
    dtype = np.dtype(dtype)
    if dtype.kind not in 'iu':
        raise TypeError("'dtype' must be an integer type")

    i = np.iinfo(dtype)
    abs_max = 2 ** (i.bits - 1)
    offset = i.min + abs_max
    return (sig * abs_max + offset).clip(i.min, i.max).astype(dtype)


def pcm2float(sig, dtype='float64'):
    """Convert PCM signal to floating point with a range from -1 to 1.
    Use dtype='float32' for single precision.
    Parameters
    ----------
    sig : array_like
        Input array, must have integral type.
    dtype : data type, optional
        Desired (floating point) data type.
    Returns
    -------
    numpy.ndarray
        Normalized floating point data.
    See Also
    --------
    float2pcm, dtype
    """
    sig = np.asarray(sig)
    if sig.dtype.kind not in 'iu':
        raise TypeError("'sig' must be an array of integers")
    dtype = np.dtype(dtype)
    if dtype.kind != 'f':
        raise TypeError("'dtype' must be a floating point type")

    i = np.iinfo(sig.dtype)
    abs_max = 2 ** (i.bits - 1)
    offset = i.min + abs_max
    return (sig.astype(dtype) - offset) / abs_max


In [228]:
fs = 8000

#os.chdir("/content/time_stretching/code/input/")

audio_file = "/content/ahh_without_silences_8k_norm.wav"
ahh, _ = librosa.load(audio_file, sr=fs, mono=True)
pcm = float2pcm(ahh)


# necesito al menos dos periodos de la señal
#200 = (50+25)+50+(50+25)
#L = 50 # muestras de la ventana
#K = L//2
# tamaño total por observacion
# total = 4*L

#quiero tener 20 milisegundos de señal
T = 20*1e-3
samples = int(T * fs) 
L = samples//4
K = L


print(pcm[50])
Audio(data=pcm, rate=fs)

1401


In [0]:
import collections
import contextlib
import sys
import wave

def read_wave(path):
    """Reads a .wav file.
    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)


class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.
    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.
    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.
    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.
    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.
    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.
    Arguments:
    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).
    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])




In [245]:
#vad 

vad = webrtcvad.Vad()

#divido el audio en frames de 20 milisegundos
frames = frame_generator(20, pcm, fs)
frames = list(frames)

#una lista para guardar los frames sonoros, otra para guardar los no-sonoros, 
#e is_frame_speech para saber si el frame original es sonoro o no (True=sonoro) 
speech = list()
not_speech = list()
is_frame_speech = list()

for frame in frames:
  if vad.is_speech(frame.bytes, fs):     #si el frame que analizo es sonoro
    speech.append(frame.bytes)                    #lo guardo en la lista correspondiente y actualizo el indice
    is_frame_speech.append(True)
  else:                                     #si el frame que analicé no era sonoro
    not_speech.append(frame.bytes)                #lo guardo en lista de no-sonoros
    is_frame_speech.append(False)


audio = list()
#verifico que no hice algo mal recomponiendo la señal
for is_speech in is_frame_speech:
  if is_speech == True:   #si el frame es sonoro
    if(speech):
      audio.append(speech[0].copy())
      speech.pop(0)
  else:
    if(not_speech):
      audio.append(not_speech[0].copy())
      not_speech.pop(0)

final = list()
for aud in audio:
  final.extend(aud)

Audio(data=final, rate=fs)

In [0]:
vad.is_speech(frames[1], fs)

In [0]:
for i, segment in enumerate(segments):
  Audio(data=segment, rate=fs)

In [0]:
a = librosa.lpc(ahh, 12)
e_ = scipy.signal.lfilter(a,[1], ahh) # obtenemos la señal error a traves del filtrado

f0,sample_f0 = get_fundamental_frequency(e_,K,L,fs)

print("Frecuencia fundamental : ",f0," ", sample_f0)
speed = 1.0
error_audio = modified_psola(e_,sample_f0, speed)

new_audio = scipy.signal.lfilter([1],a, error_audio)

#os.chdir("../output")
#librosa.output.write_wav("e_"+audio_file+f"_speed_x{speed}.wav", new_audio, fs)

Audio(data=new_audio,rate=fs)



In [244]:
a = [0, 2, 3, 2]
b = [[7],[8]]
a.append(b[0].copy())
b.pop(0)
print(b)
a


[[8]]


[0, 2, 3, 2, [7]]